In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
import pandas as pd
import numpy as np
import re
import math
import numpy as np

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/herick/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 비슷한 이름 상품 찾기
## 수학에서 distance란?
거리 = 점과 점 사이의 거리 <br/>
질문 = 두 점 사이의 거리는 무엇으로 측정 되는가? meter? feet? 혹은 각도?

### **가장 자주 쓰이는 거리(기존에 우리가 알고 있는) : Elucidian Distnace**
<img src = "image_data/eul_distance.png">
**단점**
1. 데이터의 차원이 높아질수록 거리 계산이 부정확 해진다<br/>
2. 두 점 사이의 절대값이 차이가 많이 아지만, 이러한 절대값 차이가 유의미 하지 않을때<br/>
3. 두 점의 성질이 얼마나 다른지 알고 싶을떄<br/>

### **수학에서 자주 쓰이는 :cosin similarity**
<img src = "image_data/cos_distance.svg">
<img src = "image_data/cos_distance2.png">
1. 데이터의 차원이 높아질져도 먹힌다<br/>
2. 두 점 사이으이 절대값을 중요한 정보로 여기지 않는다<br/>
3. -1 과 1 사이의 값으로 standardize 되기 때문에 스코어로 활용하기 쉽다(혹은 확률에 적용시키기 쉽다)<br/>


## 잠깜만, -1과 1 사이, 즉 스코어로 활용하기 쉽다는 것은 무엇을 의미 할까? = 확률과 연동 시키기 쉽다

case1 = 점1 주변에 다른 점들이 무작위로 찍힌다고 가정 할 때 cosin similarity 스코어가 0과 1 사이일 확률은 몇% 일까?<br/>
case2 = 점1 주변에 다른 점들이 무작위로 찍힌다고 가정 할 때 Elucidian distance가 0과 max(무작위로 찍힌 점 까지의 절대 거리 값)/2 사이일 확률은 몇% 일까?<br/>

### 실질적으로 많은 machine learning method에서 cosin similarity 를 쓴다


# 문제 : 특정 상품과 비슷한 이름을 가지는 상품을 찾으시오

In [ ]:
#https://archive.ics.uci.edu/ml/datasets/Online%20Retail
df= pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.to_excel('./data/retail_data.xlsx')

In [19]:
df = pd.read_excel('./data/retail_data.xlsx')

In [22]:
for i, x in enumerate(df['Description']):
    if i <20:
        print(x)

WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE
RED WOOLLY HOTTIE WHITE HEART.
SET 7 BABUSHKA NESTING BOXES
GLASS STAR FROSTED T-LIGHT HOLDER
HAND WARMER UNION JACK
HAND WARMER RED POLKA DOT
ASSORTED COLOUR BIRD ORNAMENT
POPPY'S PLAYHOUSE BEDROOM 
POPPY'S PLAYHOUSE KITCHEN
FELTCRAFT PRINCESS CHARLOTTE DOLL
IVORY KNITTED MUG COSY 
BOX OF 6 ASSORTED COLOUR TEASPOONS
BOX OF VINTAGE JIGSAW BLOCKS 
BOX OF VINTAGE ALPHABET BLOCKS
HOME BUILDING BLOCK WORD
LOVE BUILDING BLOCK WORD
RECIPE BOX WITH METAL HEART


In [28]:
stop_words = set(stopwords.words('english')) 
def str_filter(x):
    if type(x) == str:
        
        x = x.strip(' ').lower().replace('-', ' ').replace('/' , ' ')
        x = re.sub('[0-9]', '', x)
        x = re.sub('\b\w{1}\b', '', x)
        
        stop_words = set(stopwords.words('english'))         
        filtered_product_name_set = set([w for w in x.split(' ') if not w in stop_words] )
        return filtered_product_name_set
    else:
        return None


df['Description_filtered'] = df['Description'].apply(lambda x : str_filter(x))


In [31]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description_filtered
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,"{white, light, heart, hanging, holder}"
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{lantern, metal, white}"
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,"{coat, cream, cupid, hearts, hanger}"
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{union, flag, hot, water, knitted, bottle}"
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{red, white, woolly, hottie, heart.}"


In [34]:
target_prd = df['Description_filtered'][0]
compare_prd_list = df['Description_filtered'][1:100]

In [39]:
A = len(target_prd)
AB = 0
score_dic = dict()
for i, each_prd in enumerate(compare_prd_list):
    B = len(set(each_prd))
    for each_word in each_prd:
        if each_word in target_prd:
            AB += 1
            
    score = round(AB/(math.sqrt(A) * math.sqrt(B)), 3)
    score_dic[i] = score
    
            
            

In [44]:
for k,v in score_dic.items():
    if v > 0.8:
        print(df['Description_filtered'][k], df['Description'][k])

{'light', 'star', 'frosted', 'holder', 'glass'} GLASS STAR FROSTED T-LIGHT HOLDER
{'red', 'dot', 'hand', 'warmer', 'polka'} HAND WARMER RED POLKA DOT
{'bird', 'assorted', 'ornament', 'colour'} ASSORTED COLOUR BIRD ORNAMENT
{'bedroom', 'playhouse', "poppy's"} POPPY'S PLAYHOUSE BEDROOM 
{'playhouse', "poppy's", 'kitchen'} POPPY'S PLAYHOUSE KITCHEN
{'princess', 'charlotte', 'doll', 'feltcraft'} FELTCRAFT PRINCESS CHARLOTTE DOLL
{'', 'assorted', 'box', 'colour', 'teaspoons'} BOX OF 6 ASSORTED COLOUR TEASPOONS
{'blocks', 'vintage', 'jigsaw', 'box'} BOX OF VINTAGE JIGSAW BLOCKS 
{'alphabet', 'vintage', 'blocks', 'box'} BOX OF VINTAGE ALPHABET BLOCKS
{'building', 'home', 'block', 'word'} HOME BUILDING BLOCK WORD
{'building', 'word', 'block', 'love'} LOVE BUILDING BLOCK WORD
{'heart', 'recipe', 'metal', 'box'} RECIPE BOX WITH METAL HEART
{'england', 'new', 'doormat'} DOORMAT NEW ENGLAND
{'making', 'set', 'jars', 'jam'} JAM MAKING SET WITH JARS
{'coat', 'red', 'paris', 'fashion', 'rack'} RED CO